In [31]:
import numpy as np
import pandas as pd

### $\int_{-\inf}^{z} e^{\frac{-u^2}{2}}du = \frac{1-0.05}{2}*\sqrt{2*π}$

In [32]:
z_gamma_empty = 0.062706777

### Завдання 1

Выборочные блоки -- естественные разбиения оси $Ox$ на интервалы $ (X_{i-1},X_i],\ i=1,...,n+1\ (X_{(0)} = -\inf, X_{(n+1)} = \inf) $

Пусть $\xi_r = \xi_r(n,m) \larr$ число этих блоков, каждый из которых содержит ровно $r$ элементов второй выборки $Y=(Y_1,...,Y_m), r = 0,1,...,m$. Тогда в качестве тестовой статистики можно взять произвольную линейную комбинацию $\sum_r c_r \xi_r$. В часности, $\xi_0$ - число блоков, не содержащих ни одного элемента второй выборки, -- порождает критерий пустых блоков. 

In [94]:
# example 1
X = np.random.exponential(1, 10)
Y = np.random.exponential(1, 20)
intervals = []
for i in range(1, len(X+1)):
    intervals.append([X[i-1], X[i]])

res = np.zeros(len(intervals))
for i, x in enumerate(intervals):
    for item in Y:
        if intervals[i][0] <= item <= intervals[i][1]: 
            print(f"{intervals[i][0]} <= {item} <= {intervals[i][1]}")
            res[i] += 1
    if res[i] != 0:
        print(f"{i} :: {res[i]}")
        print("+++++++++++++++++++++++++")

print()
print()
print(res)
print("+"*20)
print(np.count_nonzero(res == 0))


0.6332036251462227 <= 0.7438379221702649 <= 0.9522865785379737
0.6332036251462227 <= 0.7900153596611136 <= 0.9522865785379737
0.6332036251462227 <= 0.8047157024232652 <= 0.9522865785379737
0.6332036251462227 <= 0.748303649761626 <= 0.9522865785379737
0 :: 4.0
+++++++++++++++++++++++++
0.11219435571890496 <= 0.7438379221702649 <= 1.8113509914661077
0.11219435571890496 <= 0.25217452908734705 <= 1.8113509914661077
0.11219435571890496 <= 0.7900153596611136 <= 1.8113509914661077
0.11219435571890496 <= 1.774361306278438 <= 1.8113509914661077
0.11219435571890496 <= 1.6073379261583827 <= 1.8113509914661077
0.11219435571890496 <= 0.1307835391823093 <= 1.8113509914661077
0.11219435571890496 <= 0.5854941026220511 <= 1.8113509914661077
0.11219435571890496 <= 0.14051157090409377 <= 1.8113509914661077
0.11219435571890496 <= 0.30409544218364876 <= 1.8113509914661077
0.11219435571890496 <= 0.35193700687294194 <= 1.8113509914661077
0.11219435571890496 <= 0.12731313096302949 <= 1.8113509914661077
0.1121

In [34]:
NM = [[500, 1000], [5000, 10000]]#, [50000, 100000]]

xi_empty = []
crit_empty = []
RES_empty = []
for nm in NM:
    
    n = nm[0]
    m = nm[1]
    ro = m/n

    X = np.random.exponential(1, n)
    Y = np.random.exponential(1, m)
    intervals = []
    for i in range(1, len(X+1)):
        intervals.append([X[i-1], X[i]])

    
    res = np.zeros(len(intervals))
    for i, x in enumerate(intervals):
        for item in Y:
            if intervals[i][0] <= item <= intervals[i][1]: res[i] += 1

    xi_empty_temp = np.count_nonzero(res == 0)
    xi_empty.append(xi_empty_temp)

    crit = n/(1 + ro) + np.math.sqrt(n)*ro*z_gamma_empty/(1+ro)**(3/2) 
    crit_empty.append(crit)
    if xi_empty_temp < crit:
        RES_empty.append("H0")
    else:
        RES_empty.append("False")


data_empty = {"xi_empty (n, m)": xi_empty, f"crit": crit_empty, "Result": RES_empty}
df_empty = pd.DataFrame.from_dict(data_empty, orient='index', columns=[f"n::{item[0]}; m::{item[1]}" for item in NM])


In [35]:
df_empty

,n::500; m::1000,n::5000; m::10000
"xi_empty (n, m)",246,2493
crit,167.206361,1668.373329
Result,False,False


## Завдання 2

### Ранжування

In [36]:
rank = lambda data : pd.Series(data).rank(method="min").to_list()

### Критерій перевірки гіпотези $H_0$


#### $\rho_n(\bar X, \bar Y) = \frac {\sum^n_{i=1} (R_i  - \bar R) (S_i - \bar S)}  {[ \sum^n_{i=1} (R_i  - \bar R)^2 \sum^m_{i=1}(S_i - \bar S)^2 ]^{\frac{1}{2}}}$

##### $\bar R = \frac {1}{n} \sum_{i=1}^n \frac{n+1}{2}$
##### $\sum^n_{i=1} (R_i  - \bar R)^2 = \frac {n(n^2-1)}{12}$


In [37]:
def Spear_Man(V):
    n = len(V)
    return 1 - 6 * np.sum((V[:, 0] - V[:, 1])**2)/(n*(n**2 -1))
    ...

In [38]:
X = [5, 2,	1,	7,	8,	4,	6,	3]
Y = [4, 3,	2,	1,	7,	6,	5,	8]


n = 8
print(n)

res = np.array([X, Y])
res = np.transpose(res)
res = res[res[:, 0].argsort()]

test = abs(Spear_Man(res))
print(test)

8
0.16666666666666663


In [60]:
z_gamma = 0.0627
N = [500, 5000, 50000]
z_gamma_spear = [z_gamma / np.math.sqrt(n) for n in N]

RHO = []
Hyp_Spear = []
for i,n in enumerate(N):
    data_xi = np.random.uniform(0,  1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi*data_nu)

    res = np.array([X, Y])
    res = np.transpose(res)
    res = res[res[:, 0].argsort()]

    test = abs(Spear_Man(res))
    RHO.append(test)
    if test < z_gamma_spear[i]:
        Hyp_Spear.append("H0")
    else:
        Hyp_Spear.append("False")

data_Spear = {"rho (X, Y)": RHO, f"z_gamma": z_gamma_spear, "Result":Hyp_Spear}
df_Spear = pd.DataFrame.from_dict(data_Spear, orient='index', columns=N)

df_Spear

,500,5000,50000
"rho (X, Y)",0.03916,0.041175,0.000865
z_gamma,0.002804,0.000887,0.00028
Result,False,False,False


Критерій Кендала

In [62]:
N = [500, 5000, 50000]
z_gamma_kend = [z_gamma / np.math.sqrt(n) /3 * 2 for n in N]
tau = []
Hyp_Kend = []
for num,n in enumerate(N):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi*data_nu)

    res = np.array([X, Y], np.int64)
    res = np.transpose(res)
    res = res[res[:, 0].argsort()]
    P_q = 0
    for i in res[1:-1, 0]:
        if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

    P_p = n*(n-1)/2 - P_q
    res = 4*P_p/(n*(n-1))-1
    tau.append(res)
    if res < z_gamma_kend[num]:
        Hyp_Kend.append("H0")
    else:
        Hyp_Kend.append("false")


data_Kend = {"tau (X, Y)": tau, f"z_gamma": z_gamma_kend, "Result": Hyp_Kend}
df_Kend = pd.DataFrame.from_dict(data_Kend, orient='index', columns=N)

df_Kend

,500,5000,50000
"tau (X, Y)",0.383535,0.391092,0.389297
z_gamma,0.001869,0.000591,0.000187
Result,false,false,false


In [41]:
data_xi = [100, 118, 112, 97,  99,  103, 102, 132, 122, 121, 115, 117, 109, 111]
data_nu = [154, 123, 120, 213, 200, 187, 155, 100, 114, 115, 107, 176, 143, 111]
n = len(data_xi)
X = rank(data_xi)
Y = rank(data_nu)

res = np.array([X, Y], np.int64)
res = np.transpose(res)
res = res[res[:, 0].argsort()]
P_q = 0
for i in res[1:-1, 0]:
    if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

P_p = n*(n-1)/2 - P_q
print(P_q, P_p, 4*P_q/(n*(n-1))-1)


15 76.0 -0.6703296703296704


### Завдання 3
#### Критерій інверсій

In [83]:

N = [500, 5000]#, 50000]
z_gamma_inv = [z_gamma / 6 * n**(3/2) for n in N]
LAMBDA = []
RES_inv = []

for num,n in enumerate(N):
    X = np.random.normal(-1, 1, n)
    k = 0
    for x in range(1, len(X)-1):
        for j in range(x, len(X)):
            i = j - 1
            if (X[j] < X[i]) and (i <  j) : k += 1

    print(k)
    temp = (abs(k-n*(n-1)/4))
    LAMBDA.append(temp)
    if temp < z_gamma_inv[num]: RES_inv.append("H0")
    else: RES_inv.append("false")

data_inv = {"LAMBDA": LAMBDA, f"z_gamma": z_gamma_inv, "Result": RES_inv}
df_inv = pd.DataFrame.from_dict(data_inv, orient='index', columns=N)

df_inv


61344
6313937


,500,5000
LAMBDA,1031.0,65187.0
z_gamma,116.834552,3694.632932
Result,false,false
